In [15]:
import os
os.chdir("C:\\Users\\Thais\\Documents\\Python\\bcb-sentiment-analysis")

import pandas as pd
import numpy as np
import scipy as sc
import pickle

DAILY_DATA = ".\\data\\financial data\\daily.xlsx"
MINUTES_DATA = ".\\data\\minutes\\copom_dates.xlsx"
MINUTES_SCORE = ".\\data\\minutes\\minutes_scores_uncased_20230423.pkl"
TARGET_INFLATION_DATA = ".\\data\\financial data\\historico_meta_inflacao.xlsx"

# 1 - Processing Data

### 1.1 Calculating Daily Historical Volatility

In [14]:
col_headers = ["21D","42D","63D","84D","105D","126D","147D","168D","189D","210D","231D","252D","273D","294D","504D","756D","1260D","1512D","1764D","2016D","2268D","2520D","2772D"]
daily_di = pd.read_excel(DAILY_DATA, sheet_name='daily_di', skiprows=5).set_index('Dates')
daily_di.columns = col_headers
daily_di.head()

,21D,42D,63D,84D,105D,126D,147D,168D,189D,210D,...,294D,504D,756D,1260D,1512D,1764D,2016D,2268D,2520D,2772D
Dates,,,,,,,,,,,,,,,,,,,,,
2001-01-01,15.7115,15.7091,15.7981,15.8141,NaN,NaN,16.1317,NaN,NaN,16.5096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-02,15.6859,15.7807,15.8498,15.8141,NaN,16.2085,16.1317,NaN,16.5291,16.5096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-03,15.6452,15.7258,15.5840,15.9106,NaN,15.8516,16.1317,NaN,16.0893,16.5096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-04,15.5716,15.5195,15.5212,15.6046,NaN,15.8243,16.1317,NaN,16.1122,16.5096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,15.5528,15.4877,15.4872,15.5659,NaN,15.8354,16.1317,NaN,16.1176,16.5096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# compute volatility using Pandas rolling and std methods, the trading days is set to 252 days
TRADING_DAYS = 252
returns = np.log(daily_di/daily_di.shift(1))
returns.fillna(0, inplace=True)
volatility_2D = returns.rolling(window=2).std()*np.sqrt(TRADING_DAYS)
volatility_1W = returns.rolling(window=5).std()*np.sqrt(TRADING_DAYS)
volatility_1M = returns.rolling(window=22).std()*np.sqrt(TRADING_DAYS)
volatility_6M = returns.rolling(window=126).std()*np.sqrt(TRADING_DAYS)
volatility_12M = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)

### 1.2 - IPCA and IPCA-15 dummies

### 1.3 - SELIC Surprise